In [1]:
# default_exp modeling.single_frame_model

In [48]:
#export
from dfdc.modeling.dataset_utils import *
from fastai.vision import *

In [3]:
data_path = Path("/home/ubuntu/data/dfdc"); print(data_path)
video_path = data_path/'dfdc_train'; print(video_path)

/home/ubuntu/data/dfdc
/home/ubuntu/data/dfdc/dfdc_train


In [4]:
train_df = create_frame_label_df(data_path, video_path, 49)
valid_df = create_frame_label_df(data_path, video_path, 48)

In [5]:
train_df.shape, valid_df.shape

((3121, 3), (2450, 3))

In [6]:
train_df.label.mean(), valid_df.label.mean()

(0.8349887856456264, 0.8277551020408164)

In [7]:
train_df.head(2)

,video_fname,frame_fnames,label
0,gdnbcgtfay,[/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfd...,1
1,klavqvgpye,[/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfd...,1


In [8]:
valid_df.head(2)

,video_fname,frame_fnames,label
0,bnfhgczmrn,[/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfd...,1
1,ojxeusdtmm,[/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfd...,1


In [19]:
flat_train_df = flatten_df(train_df)
flat_valid_df = flatten_df(valid_df)

In [20]:
flat_train_df.head(2)

,video_fname,frame_fname,label
0,gdnbcgtfay,/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfdc...,1
1,gdnbcgtfay,/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfdc...,1


In [21]:
flat_valid_df.head(2)

,video_fname,frame_fname,label
0,bnfhgczmrn,/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfdc...,1
1,bnfhgczmrn,/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfdc...,1


### 1) Single Frame Balanced Data

In [22]:
def dummy_open_image(): return torch.ones((3,5,5)).to(torch.float32)

In [23]:
#export
class SingleFrameDataset(Dataset):
    def __init__(self, flat_df):
        self.flat_df = flat_df
        
    def __getitem__(self, i):
        d1 = dict(self.flat_df.iloc[i])
        img = open_image(d1['frame_fname']).resize(299).data
        d2 = {"image":  img}
        return {**d1, **d2}
 
    def __len__(self):
        return len(self.flat_df)

In [24]:
#export
class SingleFrameSampler(Sampler):
    def __init__(self, df, flat_df):
        self.df, self.flat_df = df, flat_df

    def __iter__(self):
        # sample FAKE and REAL videos equally for each class
        class_counts = Counter(self.df['label'])
        minority_num_samples = class_counts[0]
        minority_fnames = self.df[self.df['label'] == 0]['video_fname'].values
        majority_fnames = self.df[self.df['label'] == 1].sample(minority_num_samples)['video_fname'].values
        minority_fnames, majority_fnames
        sampled_fnames = np.concatenate([minority_fnames, majority_fnames])
        sampled_df = self.df[self.df['video_fname'].isin(sampled_fnames)]
        
        # sample single frame for each sampled video
        res = []
        for _, row in sampled_df.iterrows():
            video_fname = row['video_fname']
            frame_fname = np.random.choice(row['frame_fnames'])
            res.append({'video_fname':video_fname, 'frame_fname':frame_fname})
        sampled_flat_df = pd.DataFrame(res)
        sampled_flat_df['sample_this'] = 1
        merged_flat_df = self.flat_df.merge(sampled_flat_df,on=['video_fname', 'frame_fname'],how='outer')
        
        # iterator for the epoch
        return iter(np.random.permutation(list(merged_flat_df[~merged_flat_df.sample_this.isna()].index))) 

In [25]:
train_ds = SingleFrameDataset(flat_train_df)
train_sampler = SingleFrameSampler(train_df, flat_train_df)

In [40]:
train_dl = DataLoader(train_ds, batch_size=32, sampler=train_sampler, shuffle=False)
for x in train_dl: 
#     print(x['video_fname'], x['frame_fname'], x['label'], x['image'].shape); break
    print(x['label'], x['image'].shape); break

tensor([0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
        1, 1, 0, 1, 0, 1, 1, 0]) torch.Size([32, 3, 299, 299])


In [35]:
valid_ds = SingleFrameDataset(flat_valid_df)

In [41]:
valid_dl = DataLoader(valid_ds, batch_size=32, shuffle=False)
for x in valid_dl:
#     print(x['video_fname'], x['frame_fname'], x['label'], x['image'].shape); break
    print(x['label'], x['image'].shape); break

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]) torch.Size([32, 3, 299, 299])


In [43]:
class SimpleModel(Module):
    def __init__(self):
        self.conv = nn.Conv2d(3,1,3,stride=5,dilation=3)
        self.fc = nn.Linear(3481, 2)
    def forward(self, x):
        x = self.conv(x)
        bs = x.shape[0]
        x = x.view(bs,-1)
        return self.fc(x)

In [44]:
model = SimpleModel()

In [47]:
model(x['image']).softmax(-1)[:,1].shape

torch.Size([32])

In [49]:
DataBunch??